<a href="https://colab.research.google.com/github/adityaRakhecha/secure-and-private-ai-scholarship-challenge/blob/master/Lesson-4/lesson4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch

# number of entries in our database
num_entries = 5000

# create raw database
db = torch.rand(num_entries) > 0.5
db

tensor([1, 0, 0,  ..., 0, 1, 0], dtype=torch.uint8)

**Generate Parallel Databases**

In [27]:
db[0:5]

tensor([1, 0, 0, 1, 0], dtype=torch.uint8)

In [0]:
remove_index = 2

In [0]:
# function to generate a parallel database with one entry removed
def get_parallel_db(db, remove_index):
  
  return torch.cat((db[0:remove_index],
                    db[remove_index+1:]))

In [30]:
get_parallel_db(db,2).shape

torch.Size([4999])

In [0]:
# function to generate parallel databases with one entry removed from each of the entries
def get_parallel_dbs(db):
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
pdbs = get_parallel_dbs(db)

In [33]:
len(pdbs)

5000

In [0]:
# function to create and generate parallel databases
def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(5000)

**Create database query**

In [47]:
db

tensor([1, 0, 0,  ..., 0, 1, 0], dtype=torch.uint8)

In [0]:
def query(db):
  return db.sum()

In [0]:
full_db_result = query(db)

In [0]:
sensitivity = 0
for pdb in pdbs:
  pdb_result = query(pdb)
  
  db_distance = torch.abs(pdb_result - full_db_result)
  
  if(db_distance > sensitivity):
    sensitivity = db_distance

In [53]:
sensitivity

tensor(1)

-> Calculate sensitivity for the "mean" function.

In [0]:
def cal_sensitivity(query, n_entries=1000):
  
  db, pdbs = create_db_and_parallels(n_entries)
  
  full_db_result = query(db)
  
  max_distance = 0
  for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > max_distance):
      max_distance = db_distance
      
    return max_distance

In [0]:
def query(db):
  return db.float().mean()

In [58]:
cal_sensitivity(query)

tensor(0.0005)

Calculate L1 sensitivity for threshold

In [0]:
def query(db, threshold=5):
  return (db.sum() > threshold).float()

In [64]:
for i in range(10):
  sens_f = cal_sensitivity(query, n_entries=10)
  print(sens_f)

tensor(1.)
tensor(1.)
0
tensor(1.)
0
0
0
tensor(1.)
0
0


Performing Differencing Attack

In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index=10)

In [68]:
db[10]

tensor(1, dtype=torch.uint8)

In [69]:
sum(db)

tensor(50, dtype=torch.uint8)

In [71]:
# differencing attack using sum query
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [73]:
# differencing attack using threshold
(sum(db).float() > 49) - (sum(pdb).float() > 49)

tensor(1, dtype=torch.uint8)